# California Housing Regression

## Data

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 7 columns):
housingMedianAge    20640 non-null float64
totalRooms          20640 non-null float64
totalBedrooms       20640 non-null float64
population          20640 non-null float64
households          20640 non-null float64
medianIncome        20640 non-null float64
medianHouseValue    20640 non-null float64
dtypes: float64(7)
memory usage: 1.1 MB


In [5]:
housing.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


## Exploratory Data Analysis

In [6]:
from pandas_profiling import ProfileReport

In [7]:
ProfileReport(housing, correlation_threshold=0.99) # Ignoring feature elimination

Number of variables,7
Number of observations,20640
Total Missing (%),0.0%
Total size in memory,1.1 MiB
Average record size in memory,56.0 B
Numeric,7
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## Preprocessing

### Train Test Split

In [8]:
y_val = housing['medianHouseValue']
x_data = housing.drop('medianHouseValue', axis=1)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_val, test_size=0.33)

## Scaling

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
minMaxScaler = MinMaxScaler()
# Only fit the scaler on training data.
# Transform on both
# As we are assuming that test data is unseen as of now
minMaxScaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
# scaler returns a numpy array, conver it back to a dataframe
X_train = pd.DataFrame(data=minMaxScaler.transform(X_train), 
                        columns=X_train.columns,
                        index=X_train.index)

In [14]:
# doing the same for X_test, but using scaler fitted to the training data
X_test = pd.DataFrame(data=minMaxScaler.transform(X_test),
                         columns=X_test.columns,
                         index=X_test.index)

## Creating Feature Columns for TF

In [15]:
import tensorflow as tf

In [16]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [17]:
allColumns = ['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue']
# All of these are numeric columns
numericColumns = ['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome']
# No categorical Columns
categoricalColumns = []
categoricalColumnsDimension = []
# target column
targetColumn = ['medianHouseValue']

In [18]:
# Im really proud of this method I made to automate the whole process
# of painfully creating each feature columns and then making a list for DNN
# Make sure that the target column is not in numericColumns and categoricalColumns
def getFeatureColumns(allColumns, numericColumns, categoricalColumns, 
                      categoricalColumnsDimension):
    # Make an empty list of feature columns to be filled iteratively
    feat_cols = []
    for column in allColumns:
        if column in numericColumns:
            feat_cols.append(tf.feature_column.numeric_column(column))
        elif column in categoricalColumns:
            dim = categoricalColumnsDimension[categoricalColumns.index(column)]
            tempCatCol = tf.feature_column.categorical_column_with_hash_bucket(column,
                                                                        hash_bucket_size=dim)
            # Do this only when using DNNClassr or DNNRegr
            embeddedCatCol = tf.feature_column.embedding_column(tempCatCol, 
                                                               dimension=dim)
            feat_cols.append(embeddedCatCol)
    return feat_cols

In [19]:
feat_cols = getFeatureColumns(allColumns, numericColumns, categoricalColumns, 
                              categoricalColumnsDimension)
feat_cols

[NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Input Function for the Estimator Object

In [20]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10,
                                                num_epochs=1000, shuffle=True)

## Estimator Model - DenseNeuralNetworkRegressor

In [21]:
model = tf.estimator.DNNRegressor(hidden_units=[6, 6, 6],
                                 feature_columns=feat_cols) # Because there are 6 features

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpm2j1njpg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f58011194e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training the Model

In [22]:
model.train(input_fn=input_func, max_steps=100000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpm2j1njpg/model.ckpt.
INFO:tensorflow:loss = 669648550000.0, step = 1
INFO:tensorflow:global_step/sec: 79.1588
INFO:tensorflow:loss = 314004670000.0, step = 101 (1.267 sec)
INFO:tensorflow:global_step/sec: 228.83
INFO:tensorflow:loss = 458941800000.0, step = 201 (0.436 sec)
INFO:tensorflow:global_step/sec: 147.648
INFO:tensorflow:loss = 71

INFO:tensorflow:loss = 203188760000.0, step = 6001 (0.771 sec)
INFO:tensorflow:global_step/sec: 122.293
INFO:tensorflow:loss = 191420100000.0, step = 6101 (0.833 sec)
INFO:tensorflow:global_step/sec: 118.393
INFO:tensorflow:loss = 77292044000.0, step = 6201 (0.829 sec)
INFO:tensorflow:global_step/sec: 130.194
INFO:tensorflow:loss = 83447120000.0, step = 6301 (0.768 sec)
INFO:tensorflow:global_step/sec: 141.973
INFO:tensorflow:loss = 108499090000.0, step = 6401 (0.712 sec)
INFO:tensorflow:global_step/sec: 141.459
INFO:tensorflow:loss = 70288030000.0, step = 6501 (0.700 sec)
INFO:tensorflow:global_step/sec: 151.938
INFO:tensorflow:loss = 77303250000.0, step = 6601 (0.657 sec)
INFO:tensorflow:global_step/sec: 194.216
INFO:tensorflow:loss = 29259153000.0, step = 6701 (0.515 sec)
INFO:tensorflow:global_step/sec: 146.361
INFO:tensorflow:loss = 154034860000.0, step = 6801 (0.684 sec)
INFO:tensorflow:global_step/sec: 175.136
INFO:tensorflow:loss = 126612540000.0, step = 6901 (0.576 sec)
INFO:t

INFO:tensorflow:loss = 194987460000.0, step = 13901 (0.602 sec)
INFO:tensorflow:global_step/sec: 161.367
INFO:tensorflow:loss = 61122310000.0, step = 14001 (0.620 sec)
INFO:tensorflow:global_step/sec: 174.33
INFO:tensorflow:loss = 164965350000.0, step = 14101 (0.577 sec)
INFO:tensorflow:global_step/sec: 231.058
INFO:tensorflow:loss = 40212540000.0, step = 14201 (0.434 sec)
INFO:tensorflow:global_step/sec: 160.964
INFO:tensorflow:loss = 115064120000.0, step = 14301 (0.619 sec)
INFO:tensorflow:global_step/sec: 146.783
INFO:tensorflow:loss = 148392970000.0, step = 14401 (0.680 sec)
INFO:tensorflow:global_step/sec: 192.994
INFO:tensorflow:loss = 36767674000.0, step = 14501 (0.516 sec)
INFO:tensorflow:global_step/sec: 160.106
INFO:tensorflow:loss = 83015640000.0, step = 14601 (0.630 sec)
INFO:tensorflow:global_step/sec: 159.612
INFO:tensorflow:loss = 79866100000.0, step = 14701 (0.623 sec)
INFO:tensorflow:global_step/sec: 162.963
INFO:tensorflow:loss = 102157680000.0, step = 14801 (0.614 se

INFO:tensorflow:global_step/sec: 115.793
INFO:tensorflow:loss = 39768350000.0, step = 21801 (0.858 sec)
INFO:tensorflow:global_step/sec: 153.401
INFO:tensorflow:loss = 63868207000.0, step = 21901 (0.652 sec)
INFO:tensorflow:global_step/sec: 153.572
INFO:tensorflow:loss = 104670855000.0, step = 22001 (0.653 sec)
INFO:tensorflow:global_step/sec: 161.929
INFO:tensorflow:loss = 131283200000.0, step = 22101 (0.617 sec)
INFO:tensorflow:global_step/sec: 154.83
INFO:tensorflow:loss = 214421450000.0, step = 22201 (0.645 sec)
INFO:tensorflow:global_step/sec: 192.542
INFO:tensorflow:loss = 132131970000.0, step = 22301 (0.525 sec)
INFO:tensorflow:global_step/sec: 146.411
INFO:tensorflow:loss = 112897730000.0, step = 22401 (0.677 sec)
INFO:tensorflow:global_step/sec: 146.546
INFO:tensorflow:loss = 35681400000.0, step = 22501 (0.682 sec)
INFO:tensorflow:global_step/sec: 154.959
INFO:tensorflow:loss = 63041618000.0, step = 22601 (0.645 sec)
INFO:tensorflow:global_step/sec: 122.806
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 164.937
INFO:tensorflow:loss = 138020060000.0, step = 29701 (0.609 sec)
INFO:tensorflow:global_step/sec: 154.593
INFO:tensorflow:loss = 83782205000.0, step = 29801 (0.643 sec)
INFO:tensorflow:global_step/sec: 147.969
INFO:tensorflow:loss = 53641690000.0, step = 29901 (0.675 sec)
INFO:tensorflow:global_step/sec: 191.394
INFO:tensorflow:loss = 66346180000.0, step = 30001 (0.523 sec)
INFO:tensorflow:global_step/sec: 161.988
INFO:tensorflow:loss = 134431244000.0, step = 30101 (0.617 sec)
INFO:tensorflow:global_step/sec: 151.196
INFO:tensorflow:loss = 64921125000.0, step = 30201 (0.661 sec)
INFO:tensorflow:global_step/sec: 204.68
INFO:tensorflow:loss = 72160035000.0, step = 30301 (0.496 sec)
INFO:tensorflow:global_step/sec: 154.821
INFO:tensorflow:loss = 168085180000.0, step = 30401 (0.642 sec)
INFO:tensorflow:global_step/sec: 165.223
INFO:tensorflow:loss = 143861170000.0, step = 30501 (0.603 sec)
INFO:tensorflow:global_step/sec: 188.616
INFO:tensorflow:loss

...

INFO:tensorflow:global_step/sec: 167.851
INFO:tensorflow:loss = 98092745000.0, step = 61301 (0.594 sec)
INFO:tensorflow:global_step/sec: 110.496
INFO:tensorflow:loss = 144230480000.0, step = 61401 (0.916 sec)
INFO:tensorflow:global_step/sec: 153.728
INFO:tensorflow:loss = 82108670000.0, step = 61501 (0.640 sec)
INFO:tensorflow:global_step/sec: 122.81
INFO:tensorflow:loss = 136089060000.0, step = 61601 (0.816 sec)
INFO:tensorflow:global_step/sec: 188.135
INFO:tensorflow:loss = 74169040000.0, step = 61701 (0.530 sec)
INFO:tensorflow:global_step/sec: 153.641
INFO:tensorflow:loss = 54591988000.0, step = 61801 (0.667 sec)
INFO:tensorflow:global_step/sec: 157.579
INFO:tensorflow:loss = 132722840000.0, step = 61901 (0.619 sec)
INFO:tensorflow:global_step/sec: 157.638
INFO:tensorflow:loss = 130790350000.0, step = 62001 (0.645 sec)
INFO:tensorflow:global_step/sec: 141.139
INFO:tensorflow:loss = 30252216000.0, step = 62101 (0.704 sec)
INFO:tensorflow:global_step/sec: 210.085
INFO:tensorflow:loss

INFO:tensorflow:global_step/sec: 167.676
INFO:tensorflow:loss = 95040420000.0, step = 69201 (0.595 sec)
INFO:tensorflow:global_step/sec: 185.548
INFO:tensorflow:loss = 128668530000.0, step = 69301 (0.537 sec)
INFO:tensorflow:global_step/sec: 155.778
INFO:tensorflow:loss = 26299615000.0, step = 69401 (0.642 sec)
INFO:tensorflow:global_step/sec: 149.462
INFO:tensorflow:loss = 195591390000.0, step = 69501 (0.667 sec)
INFO:tensorflow:global_step/sec: 160.467
INFO:tensorflow:loss = 79486320000.0, step = 69601 (0.624 sec)
INFO:tensorflow:global_step/sec: 165.772
INFO:tensorflow:loss = 138601960000.0, step = 69701 (0.606 sec)
INFO:tensorflow:global_step/sec: 149.326
INFO:tensorflow:loss = 74453836000.0, step = 69801 (0.667 sec)
INFO:tensorflow:global_step/sec: 141.43
INFO:tensorflow:loss = 90417545000.0, step = 69901 (0.707 sec)
INFO:tensorflow:global_step/sec: 167.933
INFO:tensorflow:loss = 124212490000.0, step = 70001 (0.594 sec)
INFO:tensorflow:global_step/sec: 158.031
INFO:tensorflow:loss

INFO:tensorflow:global_step/sec: 155.359
INFO:tensorflow:loss = 49096966000.0, step = 77101 (0.643 sec)
INFO:tensorflow:global_step/sec: 198.025
INFO:tensorflow:loss = 93553000000.0, step = 77201 (0.507 sec)
INFO:tensorflow:global_step/sec: 179.737
INFO:tensorflow:loss = 36650090000.0, step = 77301 (0.560 sec)
INFO:tensorflow:global_step/sec: 171.917
INFO:tensorflow:loss = 79307870000.0, step = 77401 (0.586 sec)
INFO:tensorflow:global_step/sec: 196.776
INFO:tensorflow:loss = 184333900000.0, step = 77501 (0.499 sec)
INFO:tensorflow:global_step/sec: 190.072
INFO:tensorflow:loss = 52904993000.0, step = 77601 (0.533 sec)
INFO:tensorflow:global_step/sec: 143.582
INFO:tensorflow:loss = 83923680000.0, step = 77701 (0.692 sec)
INFO:tensorflow:global_step/sec: 166.891
INFO:tensorflow:loss = 96579790000.0, step = 77801 (0.594 sec)
INFO:tensorflow:global_step/sec: 220.427
INFO:tensorflow:loss = 100200150000.0, step = 77901 (0.462 sec)
INFO:tensorflow:global_step/sec: 166.736
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 173.092
INFO:tensorflow:loss = 40657110000.0, step = 85001 (0.576 sec)
INFO:tensorflow:global_step/sec: 157.068
INFO:tensorflow:loss = 28852773000.0, step = 85101 (0.635 sec)
INFO:tensorflow:global_step/sec: 146.197
INFO:tensorflow:loss = 42760210000.0, step = 85201 (0.693 sec)
INFO:tensorflow:global_step/sec: 105.397
INFO:tensorflow:loss = 12886490000.0, step = 85301 (0.945 sec)
INFO:tensorflow:global_step/sec: 166.558
INFO:tensorflow:loss = 55775126000.0, step = 85401 (0.595 sec)
INFO:tensorflow:global_step/sec: 179.71
INFO:tensorflow:loss = 119604850000.0, step = 85501 (0.562 sec)
INFO:tensorflow:global_step/sec: 202.494
INFO:tensorflow:loss = 167518320000.0, step = 85601 (0.489 sec)
INFO:tensorflow:global_step/sec: 178.822
INFO:tensorflow:loss = 76869980000.0, step = 85701 (0.562 sec)
INFO:tensorflow:global_step/sec: 249.196
INFO:tensorflow:loss = 94311530000.0, step = 85801 (0.401 sec)
INFO:tensorflow:global_step/sec: 165.885
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 273.936
INFO:tensorflow:loss = 74315500000.0, step = 92901 (0.370 sec)
INFO:tensorflow:global_step/sec: 168.439
INFO:tensorflow:loss = 82541250000.0, step = 93001 (0.589 sec)
INFO:tensorflow:global_step/sec: 153.013
INFO:tensorflow:loss = 60934066000.0, step = 93101 (0.652 sec)
INFO:tensorflow:global_step/sec: 201.286
INFO:tensorflow:loss = 68424480000.0, step = 93201 (0.496 sec)
INFO:tensorflow:global_step/sec: 158.205
INFO:tensorflow:loss = 104824580000.0, step = 93301 (0.634 sec)
INFO:tensorflow:global_step/sec: 130.144
INFO:tensorflow:loss = 152554050000.0, step = 93401 (0.768 sec)
INFO:tensorflow:global_step/sec: 164.13
INFO:tensorflow:loss = 56596767000.0, step = 93501 (0.609 sec)
INFO:tensorflow:global_step/sec: 159.012
INFO:tensorflow:loss = 52876603000.0, step = 93601 (0.636 sec)
INFO:tensorflow:global_step/sec: 148.199
INFO:tensorflow:loss = 186414580000.0, step = 93701 (0.667 sec)
INFO:tensorflow:global_step/sec: 172.722
INFO:tensorflow:loss 

## Making predictions

### Making prediction input function for the DNN model object

In [23]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=None, batch_size=10,
                                                        num_epochs=1, shuffle=False)

### Making a prediction generator object and generating a list of predicitons

In [24]:
prediction_generator = model.predict(input_fn=predict_input_func)
predictions = list(prediction_generator)
predictions[0] # each tensorflow prediction is a dictionary value

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpm2j1njpg/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'predictions': array([283843.], dtype=float32)}

### Calculating the RMSE of predictions manually

In [25]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])
final_preds[0] # now it is an array

array([283843.], dtype=float32)

In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, final_preds)**0.5 # root of MSE is RMSE, Duh!

92789.33727261542

## Evaluating the model against TestSet

In [28]:
test_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=10,
                                                     num_epochs=1, shuffle=False)
model.evaluate(input_fn=test_input_func, name='Stock Test Set Evaluation')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-20T13:45:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpm2j1njpg/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-20-13:45:33
INFO:tensorflow:Saving dict for global step 100000: average_loss = 8609866000.0, global_step = 100000, label/mean = 208071.5, loss = 85997670000.0, prediction/mean = 212273.61
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100000: /tmp/tmpm2j1njpg/model.ckpt-100000


{'average_loss': 8609866000.0,
 'label/mean': 208071.5,
 'loss': 85997670000.0,
 'prediction/mean': 212273.61,
 'global_step': 100000}

Model ain't overfitting by the looks of it